<a href="https://colab.research.google.com/github/Viniciusalgueiro/python_Studing/blob/main/churn_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**`Importanção de bibliotecas`**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

**Importanção de arquivos**

In [ ]:
from google.colab import files
arquivo = files.upload()

In [ ]:
df = pd.read_csv('Telco-Customer-Churn.csv')

In [ ]:
df_val = df.iloc[:10]

df_val

In [ ]:
#Podemos retirar a coluna de id
df.drop('customerID', 1, inplace=True)

In [ ]:
#verifica os tipos
df.dtypes


In [ ]:
#é possivel usar 'coerce' mas o valores que tem espço em branco vão ficar vazio, podemos retirá-los
df.TotalCharges = pd.to_numeric(df.TotalCharges, errors='coerce')

df.TotalCharges.isna()

In [ ]:
#Vamos ver existem campos que estão com campos vazios
df[df.TotalCharges.isna()]

In [ ]:
#Podemos excluir os que estão vazios
df.dropna(subset=['TotalCharges'], inplace=True)

df


# **Grafico para analise de Tenure ( total de meses do usuario usando o seriviço)**


In [ ]:
#Graficos para explorar os dados
#filtrar por churn no e yes e verificar os tenure
dfno = df[df.Churn=='No'].tenure
dfyes = df[df.Churn=='Yes'].tenure

plt.hist([dfno,dfyes], color=['blue', 'red'], label=['Churn = no', 'Churn = yes'])
plt.legend()

In [ ]:
#Graficos para explorar os dados
#filtrar por churn no e yes e verificar os Dependents
dfno = df[df.Churn=='No'].Dependents
dfyes = df[df.Churn=='Yes'].Dependents

plt.hist([dfno,dfyes], color=['blue', 'red'], label=['Churn = no', 'Churn = yes'])
plt.legend()


In [ ]:

#Graficos para explorar os dados
#filtrar por churn no e yes e verificar os MonthlyCharges
dfno = df[df.Churn=='No'].MonthlyCharges
dfyes = df[df.Churn=='Yes'].MonthlyCharges

plt.hist([dfno,dfyes], color=['blue', 'red'], label=['Churn = no', 'Churn = yes'])
plt.legend()

In [ ]:
#Verificar quão balanceado esta o dataset
dfno = df[df.Churn=='No'].Churn
dfyes = df[df.Churn=='Yes'].Churn

total= dfno.count() + dfyes.count()
percent = round ((dfyes.count() / total) * 100 , 2)

print(f'YES: {dfno.count()}', f'No: {dfyes.count()}', f'% No: {percent}')

plt.hist([dfno,dfyes], color=['blue', 'red'], label=['Churn = no', 'Churn = yes'])
plt.legend()


In [ ]:
#Ajustas os camp que tem "No service" para No
for i in df:  
  if df[i].dtypes=='object':
    print(f'{i} : {df[i].unique()}')

df.replace('No internet service', 'No', inplace=True)
df.replace('No phone service', 'No', inplace=True)

df

In [ ]:
#O que precisamos fazer agora e colocar os campos categoricos como numericos usando o hot enconding do panadas.
#Vamosa fazer essa tranformação pois pois é muito mais facil para os modelos trabalharam com valores, pois são modelos matematicos

dfnum= pd.get_dummies(data=df, columns=['gender','InternetService', 'Partner','Dependents','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod'])
dfnum.columns


In [ ]:

#Podemos ver um example dos valores codificados
dfnum.sample(5)


In [ ]:
#Label encoder coluna Churn
le = LabelEncoder()
dfnum['Churn'] = le.fit_transform(dfnum['Churn'])
dfnum.Churn.sample(15)


In [ ]:
#Fazer o scale dos dados nas colunas de "tenure", monthly charges e total cherges'

scaler = MinMaxScaler()
dfnum[['tenure','MonthlyCharges','TotalCharges']] = scaler.fit_transform(dfnum[['tenure','MonthlyCharges','TotalCharges']])
dfnum.sample(5)


In [ ]:
#Separar os dados em features e labels
features = dfnum.drop('Churn', 1)
label = dfnum['Churn']
print(label)

**Parte pra valiação**

In [ ]:
#Pegar uam parte para validação
val = features.head(10)
val

In [ ]:
#Novo featured
features = features.iloc[10:]
features



In [ ]:
label = label.iloc[10:]
label


In [ ]:
#Separa train test e split
X_Train, X_test, y_train, y_test = train_test_split(features,label,test_size=0.2, random_state=42)
print(X_Train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# **Importando Tensorflow e keras**

In [ ]:
#impotar tensor flow e keras

model = Sequential([
  Dense(20, input_shape=(41,), activation='relu'),
  Dense(52, activation='relu'),
  Dense(1,activation='sigmoid')
])

#Passando parametro de execução
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  

In [ ]:
#Um detalhe importante a ser mencionado,e que estamos passando as camadas da rede neural, e na o primieiro comando na verdade já é a segunda camada
# e neste caso estamos passando 20 neuronio(valor arbitrário) e o segundo parêmtro é a uqbntidade de entradas, que no nosso caso são 26 campos e a função
#de ativação usada no neurônio que neste caso é a relu.

#ReLU é a função de ativação mais amplamente utilizada ao projetar redes neurais atualmente.
#Primeiramente, a função ReLU é não linear, o que significa que podemos facilmente copiar os erros para trás e ter várias camadas de 
#neurônios ativados pela função ReLU.

#A principal vantagem de usar a função ReLU sobre outras funções de ativação é que ela não ativa todos os neurônios ao mesmo tempo.

#Podemos ver como foi criado nosso modelo


In [ ]:
#Treinamento
model.summary()

In [ ]:
model.fit(X_Train,y_train, epochs=200)

In [ ]:
pred_val = model.predict(val)
print(pred_val)

In [ ]:
y_pred = []
for i in pred_val:
  if i > 0.5:
    y_pred.append(1)
  else:
    y_pred.append(0)

cliente = df_val['customerID']
real = df_val['Churn']
pred = list(pred_val.flatten())

df=pd.DataFrame({'cliente':cliente, 'previsao':pred, 'real':real, 'churn':y_pred})

print(df)

In [ ]:
import seaborn as sns

df = pd.DataFrame(df, columns=['real','churn'])
confusion_matrix = pd.crosstab(df['real'], df['churn'], rownames=['real'], colnames=['churn'])

sns.heatmap(confusion_matrix, annot=True)

In [ ]:
model.evaluate(X_test,y_test)